# Machine Learning Algorithms for Predicting Client Churn (Prototype)

> Disclaimer: The data presented here is sensitive and for internal use only

TODO: Description

## How To

## Technologies

- Jupyter Notebooks
- Python3
  - Pandas
 
## Calculate

### Libraries

In [1]:
import pandas as pd

In [2]:
!pip install scipy

In [3]:
!pip install sklearn

In [4]:
from sklearn import preprocessing

### Data

#### Raw Data

In [5]:
raw_data = pd.read_csv('./datasets/sample-raw-data.csv')

The raw data needs to be "cleaned" before a machine learning algorithm can be performed on it. MUST HAVE steps to be taken are

- Concatenate data based on each unique client
- Remove columns that are clearly not great features choices

Further steps that can be taken:

- Separate "cohorts" based on purchase date, or normalize older and newer clients versus each other
  - Explanation: Because we're trying to predict the likelihood of cancellation, newer clients will naturally (and mistakenly) appear to cancel at a lower rate than older clients; this is because older clients have been given more "opportunity" to cancel
- Convert dates to a simple scalar value
- Convert location addresses to scalar or vector values; i.e., avoid (or be explicit about) basing the prediction on simple population density
- Normalize each scalar value

In [6]:
# Current "dirty" data
raw_data.head(5)

,CHURN_DATE,HEALTH_SCORE,HEALTH_SCORE_UPDATED_DATE,RELATIONSHIP_STRENGTH,SENTIMENT,LAST_ZENDESK_TICKET_DATE,NPS_SCORE,NPS_SCORE_DATE,LAST_LOGIN,IS_EDITED,...,LAUNCH_DATE,PAYMENT_PLAN,FREQUENCY,SUBSCRIPTION_ID,SUBSCRIPTION_ITEM_ID,PRODUCT_ID,PRODUCT_QUANTITY,PRODUCT_PRICE,START_DATE,END_DATE
0,NaN,73.0,2019-11-08 00:31:00.000 +0000,Good,neutral,2019-11-05 19:54:48.000 +0000,10.0,2019-04-29 04:00:00.000 +0000,2019-11-18 05:00:00.000 +0000,True,...,2019-04-15 00:00:00.000 +0000,Monthly,Monthly,sub_E3cpvGPbhWYXL9,si_E3cpsucoXGLgRq,addon_catering_store_monthly,1,139.0,2018-11-28 16:07:31.000 +0000,NaN
1,NaN,73.0,2019-11-08 00:31:00.000 +0000,Good,neutral,2019-11-05 19:54:48.000 +0000,10.0,2019-04-29 04:00:00.000 +0000,2019-11-18 05:00:00.000 +0000,True,...,2019-04-15 00:00:00.000 +0000,Monthly,Monthly,sub_E3cpvGPbhWYXL9,si_E3cpSjCw4yRknW,base_plus_monthly,1,199.0,2018-11-28 16:07:31.000 +0000,NaN
2,NaN,50.0,2019-11-08 00:31:00.000 +0000,Neutral,NaN,NaN,NaN,NaN,2019-11-18 05:00:00.000 +0000,False,...,2016-10-31 00:00:00.000 +0000,Monthly,Monthly,sub_9OtLNjGdGik2ge,si_FkYuZKZWOAhqdC,base_plus_monthly,1,199.0,2019-09-04 19:05:07.000 +0000,NaN
3,NaN,50.0,2019-11-08 00:31:00.000 +0000,Neutral,NaN,NaN,NaN,NaN,2019-11-18 05:00:00.000 +0000,False,...,2016-10-31 00:00:00.000 +0000,Monthly,Monthly,sub_9OtLNjGdGik2ge,si_1965Ys2vS2BuzRKbWsoEErwQ,baby,1,99.0,2016-10-18 21:55:55.000 +0000,2019-09-18 21:55:54.000 -0700
4,NaN,86.0,2019-11-08 00:31:00.000 +0000,Good,neutral,2019-07-29 17:01:59.000 +0000,NaN,NaN,2019-07-24 04:00:00.000 +0000,True,...,2017-10-24 00:00:00.000 +0000,Monthly,Monthly,sub_BGEa6oquQ3Wrhh,si_1Ati6w2vS2BuzRKbP1djq3kT,plus,1,199.0,2017-08-22 19:32:26.000 +0000,NaN


In [7]:
raw_data.count()

CHURN_DATE                           160
HEALTH_SCORE                        4538
HEALTH_SCORE_UPDATED_DATE           4875
RELATIONSHIP_STRENGTH               4993
SENTIMENT                           1778
LAST_ZENDESK_TICKET_DATE            2404
NPS_SCORE                            950
NPS_SCORE_DATE                       942
LAST_LOGIN                          4919
IS_EDITED                           5814
IS_DELINQUENT                       5814
IS_CONCEPT                          5814
NUMBER_OF_CONCEPTS                  2442
NUMBER_OF_LOCATIONS                 5636
ANNUAL_REVENUE                      4954
ANNUAL_HOSPITALITY_GROUP_REVENUE    5814
PARTNERSHIP                         1878
BILLING_CITY                        5795
BILLING_COUNTRY                     5810
BILLING_STATE                       5777
BUSINESS_MARKET                     4115
BUSINESS_TYPE                       5793
SERVICE_STYLE                       5790
NEW_OR_EXISTING                     5790
CUISINE         

#### Filtered columns

The following dataset concatenates duplicate clients and filters in only the columns we'll use for prototype.

In a live environment, these actions can be performed programmatically.

In [8]:
filtered_data = pd.read_csv('./datasets/sample-filtered-data.csv')

In [9]:
filtered_data.head(5)

,SUBSCRIPTION_ID,RELATIONSHIP_STRENGTH,SENTIMENT,NPS_SCORE,IS_DELINQUENT,NUMBER_OF_LOCATIONS,ANNUAL_REVENUE,PARTNERSHIP,BILLING_CITY,BUSINESS_TYPE,CUISINE,PAYMENT_PLAN,CHURN_DATE
0,sub_E3cpvGPbhWYXL9,Good,neutral,10.0,False,1.0,4056.0,NaN,Philadelphia,Restaurant,American (Traditional),Monthly,NaN
1,sub_9OtLNjGdGik2ge,Neutral,NaN,NaN,False,1.0,1188.0,NaN,New York,Restaurant,American (New),Monthly,NaN
2,sub_BGEa6oquQ3Wrhh,Good,neutral,NaN,False,1.0,1908.0,US Foods,South Bend,Restaurant,American (Traditional),Monthly,NaN
3,sub_EtPCVOt1zlHjQj,Good,NaN,NaN,False,1.0,1188.0,US Foods,Walhalla,Restaurant,"[""Diner"",""Sandwiches"",""American"",""Frozen Yogur...",NaN,NaN
4,sub_EWvgOoZikMZEzh,Neutral,neutral,NaN,False,3.0,2388.0,NaN,Vancouver,Restaurant,Other,Monthly,NaN


In [10]:
filtered_data.count()

SUBSCRIPTION_ID          4638
RELATIONSHIP_STRENGTH    4019
SENTIMENT                1363
NPS_SCORE                 731
IS_DELINQUENT            4638
NUMBER_OF_LOCATIONS      4498
ANNUAL_REVENUE           3972
PARTNERSHIP              1645
BILLING_CITY             4621
BUSINESS_TYPE            4621
CUISINE                  4619
PAYMENT_PLAN             4473
CHURN_DATE                 96
dtype: int64

#### Further cleaning

In [11]:
current_data = pd.DataFrame(filtered_data).copy(deep=True) 

In [12]:
# Show column names
print(current_data.columns.values)

['SUBSCRIPTION_ID' 'RELATIONSHIP_STRENGTH' 'SENTIMENT' 'NPS_SCORE'
 'IS_DELINQUENT' 'NUMBER_OF_LOCATIONS' 'ANNUAL_REVENUE' 'PARTNERSHIP'
 'BILLING_CITY' 'BUSINESS_TYPE' 'CUISINE' 'PAYMENT_PLAN' 'CHURN_DATE']


##### Handle the "feelings" columns

In [13]:
def clean_string(x):
    if isinstance(x, str):
        return x.lower()
    return None

In [14]:
def handle_feelings(x):
    cleaned = clean_string(x)
    if cleaned == 'good' or cleaned == 'positive':
        return 1
    if cleaned == 'neutral':
        return 0
    if cleaned == 'bad' or cleaned == 'negative':
        return -1
    return None

In [15]:
current_data['RELATIONSHIP_STRENGTH'] = current_data['RELATIONSHIP_STRENGTH'].apply(handle_feelings)

In [16]:
current_data['SENTIMENT'] = current_data['SENTIMENT'].apply(handle_feelings)

##### Normalize the scalar columns

In [17]:
def normalize(df):
    return (df - df.mean()) / df.std()

In [18]:
current_data['NPS_SCORE'] = normalize(current_data['NPS_SCORE'])

In [19]:
current_data['NUMBER_OF_LOCATIONS'] = normalize(current_data['NUMBER_OF_LOCATIONS'])

In [20]:
current_data['ANNUAL_REVENUE'] = normalize(current_data['ANNUAL_REVENUE'])